In [1]:
import json
import pandas as pd
import glob

files = glob.glob('/Users/petebrown/Developer/data-updater-v2/bbc-json/lineups/*.json')

files = sorted(files)

files[:3]

['/Users/petebrown/Developer/data-updater-v2/bbc-json/lineups/2020-01-01.json',
 '/Users/petebrown/Developer/data-updater-v2/bbc-json/lineups/2020-01-04.json',
 '/Users/petebrown/Developer/data-updater-v2/bbc-json/lineups/2020-01-07.json']

In [2]:
def extract_date(filename):
    return filename.split('/')[-1].split('.')[0]

dates = [extract_date(file) for file in files]

dates[:3]

['2020-01-01', '2020-01-04', '2020-01-07']

In [3]:
all_officials_df = pd.DataFrame()

for file in files:
    game_date = extract_date(file)

    with open(file) as f:
        data = json.load(f)

    officials = data['officials']

    officials_df = pd.DataFrame(officials)

    officials_df['game_date'] = game_date

    all_officials_df = pd.concat([all_officials_df, officials_df])

all_officials_df

,id,type,lastName,firstName,shortLastName,shortFirstName,game_date,opId
0,urn:bbc:sportsdata:football:official:d4vllv2sd...,Referee,Finnie,William James,Finnie,Will,2020-01-01,NaN
1,urn:bbc:sportsdata:football:official:829eow9y1...,Assistant referee 1,Simpson,Joe,Simpson,Joe,2020-01-01,NaN
2,urn:bbc:sportsdata:football:official:7zq7waqh3...,Assistant referee 2,Gooch,Peter,Gooch,Peter,2020-01-01,NaN
3,urn:bbc:sportsdata:football:official:dyhw9jbfd...,Fourth official,Smith,Lewis,Smith,Lewis,2020-01-01,NaN
0,urn:bbc:sportsdata:football:official:3ozjhxff4...,Referee,Scott,Graham,Scott,Graham,2020-01-04,NaN
...,...,...,...,...,...,...,...,...
3,urn:bbc:sportsdata:football:official:b4aluow6i...,Fourth official,Khan,Abbas,Khan,Abbas,2025-04-18,NaN
0,urn:bbc:sportsdata:football:official:23fcti22n...,Referee,Kirk,Thomas,Kirk,Thomas,2025-04-21,NaN
1,urn:bbc:sportsdata:football:official:c1s8pdzwl...,Assistant referee 1,Graham,Jacob,Graham,Jacob,2025-04-21,NaN
2,urn:bbc:sportsdata:football:official:eg4lszrjq...,Assistant referee 2,Cooke,Thomas,Cooke,Thomas,2025-04-21,NaN


In [51]:
all_home_df = pd.DataFrame()

all_home_starters_df = pd.DataFrame()

all_home_subs_df = pd.DataFrame()

all_home_pos_df = pd.DataFrame()

for file in files:
    game_date = extract_date(file)

    team = 'home'

    with open(file) as f:
        data = json.load(f)

    home = data['homeTeam']

    home_df = pd.json_normalize(home)

    home_df['game_date'] = game_date

    home_df['team'] = team

    all_home_df = pd.concat([all_home_df, home_df])

    home_starters_df = home_df['players.starters'][0].copy()

    home_starters_df = pd.json_normalize(home_starters_df)

    home_starters_df['game_date'] = game_date

    home_starters_df['team'] = team

    all_home_starters_df = pd.concat([all_home_starters_df, home_starters_df])

    home_subs_df = home_df['players.substitutes'][0].copy()

    home_subs_df = pd.json_normalize(home_subs_df)

    home_subs_df['game_date'] = game_date

    home_subs_df['team'] = team

    all_home_subs_df = pd.concat([all_home_subs_df, home_subs_df])

    if 'pitchLayout' in home_df.keys():
        home_pos = pd.json_normalize(home_df['pitchLayout'][0])

        home_pos_df = pd.DataFrame()
        for pos in range(len(home_pos)):
            pos_df = pd.json_normalize(home_df['pitchLayout'][0][pos])
            home_pos_df = pd.concat([home_pos_df, pos_df])
            home_pos_df['game_date'] = game_date
            home_pos_df['team'] = team
            all_home_pos_df = pd.concat([all_home_pos_df, home_pos_df]).reset_index(drop=True).drop_duplicates()

# home_df = all_home_df[['game_date', 'id', 'urn', 'alignment', 'name.fullName', 'name.shortName', 'name.code', 'formation.layout', 'formation.value', 'formation.accessible', 'manager.id', 'manager.name.full']].copy()

all_home_pos_df.head(15)

,shirtNumber,name,accessibleText,game_date,team
0,1,McGee,"1, McGee, Goalkeeper",2024-10-01,home
1,22,O'Connor,"22, O'Connor, Defender",2024-10-01,home
2,5,Davies,"5, Davies, Defender",2024-10-01,home
3,6,Turnbull,"6, Turnbull, Defender",2024-10-01,home
4,23,Wood,"23, Wood, Defender",2024-10-01,home
5,16,Merrie,"16, Merrie, Defensive Midfielder",2024-10-01,home
6,30,Patrick,"30, Patrick, Midfielder",2024-10-01,home
7,18,Jennings,"18, Jennings, Midfielder",2024-10-01,home
8,8,Hendry,"8, Hendry, Midfielder",2024-10-01,home
9,19,Saunders,"19, Saunders, Midfielder",2024-10-01,home


In [50]:
all_away_df = pd.DataFrame()

all_away_starters_df = pd.DataFrame()

all_away_subs_df = pd.DataFrame()

all_away_pos_df = pd.DataFrame()

for file in files:
    game_date = extract_date(file)

    team = 'away'

    with open(file) as f:
        data = json.load(f)

    away = data['awayTeam']

    away_df = pd.json_normalize(away)

    away_df['game_date'] = game_date

    away_df['team'] = team

    all_away_df = pd.concat([all_away_df, away_df])

    away_starters_df = away_df['players.starters'][0].copy()

    away_starters_df = pd.json_normalize(away_starters_df)

    away_starters_df['game_date'] = game_date

    away_starters_df['team'] = team

    all_away_starters_df = pd.concat([all_away_starters_df, away_starters_df])

    away_subs_df = away_df['players.substitutes'][0].copy()

    away_subs_df = pd.json_normalize(away_subs_df)

    away_subs_df['game_date'] = game_date

    away_subs_df['team'] = team

    all_away_subs_df = pd.concat([all_away_subs_df, away_subs_df])

    if 'pitchLayout' in away_df.keys():
        away_pos = pd.json_normalize(away_df['pitchLayout'][0])

        away_pos_df = pd.DataFrame()
        for pos in range(len(away_pos)):
            pos_df = pd.json_normalize(away_df['pitchLayout'][0][pos])
            away_pos_df = pd.concat([away_pos_df, pos_df])
            away_pos_df['game_date'] = game_date
            away_pos_df['team'] = team
            all_away_pos_df = pd.concat([all_away_pos_df, away_pos_df]).reset_index(drop=True).drop_duplicates()

# away_df = all_away_df[['game_date', 'id', 'urn', 'alignment', 'name.fullName', 'name.shortName', 'name.code', 'formation.layout', 'formation.value', 'formation.accessible', 'manager.id', 'manager.name.full']].copy()

all_away_pos_df.head(15)

,shirtNumber,name,accessibleText,game_date,team
0,12,Barden,"12, Barden, Goalkeeper",2024-10-01,away
1,2,Sobowale,"2, Sobowale, Defender",2024-10-01,away
2,5,Wright,"5, Wright, Defender",2024-10-01,away
3,20,Freckleton,"20, Freckleton, Defender",2024-10-01,away
4,17,King,"17, King, Midfielder",2024-10-01,away
5,7,Cotterill,"7, Cotterill, Midfielder",2024-10-01,away
6,18,Kilkenny,"18, Kilkenny, Midfielder",2024-10-01,away
7,27,Cox,"27, Cox, Midfielder",2024-10-01,away
8,22,Butterworth,"22, Butterworth, Attacking Midfielder",2024-10-01,away
9,21,Tshimanga,"21, Tshimanga, Striker",2024-10-01,away


In [87]:
games_df = pd.concat([all_home_df, all_away_df]) \
    .sort_values(['game_date', 'alignment'], ascending=[True, False]) \
    .drop(columns={
        'players.starters',
        'players.substitutes',
        'pitchLayout'
    }) \
    .reset_index(drop=True)

starters_df = pd.concat([all_home_starters_df, all_away_starters_df]) \
    .drop(columns={
        'urn',
        'name.short',
        'cards',
        'substitutes',
        'substitutedOff.playerOnUrn'
    }) \
    .reset_index(drop=True)

subs_df = pd.concat([all_home_subs_df, all_away_subs_df]).reset_index(drop=True)

pos_df = pd.concat([all_home_pos_df, all_away_pos_df]).reset_index(drop=True)

In [88]:
starters_df.iloc[-1]

position                                    Striker
shirtNumber                                      14
formationPlace                                    9
isCaptain                                     False
displayName                               K. Dennis
name.first                                 Kristian
name.last                                    Dennis
card.type.value                                 NaN
card.type.accessibleValue                       NaN
card.time.value                                 NaN
card.time.accessibleValue                       NaN
substitutedOff.periodId                         2.0
substitutedOff.timeMin                         66.0
substitutedOff.timeMinSec                     65:54
substitutedOff.timestamp       2025-04-21T15:22:53Z
substitutedOff.playerOnName             C. Jennings
substitutedOff.reason                      Tactical
game_date                                2025-04-21
team                                           away
name.shirt  

In [102]:
cards_df = starters_df[['game_date', 'team', 'shirtNumber', 'displayName', 'card.type.value', 'card.time.value']].dropna()

cards_df['card_min'] = cards_df['card.time.value'].str.split('\'').str[0]

cards_df['card_inj_time'] = cards_df['card.time.value'].str.split('+').str[1]

cards_df[cards_df['game_date']=='2020-03-07']

,game_date,team,shirtNumber,displayName,card.type.value,card.time.value,card_min,card_inj_time
157,2020-03-07,home,18,H. Rodgers,second-yellow,90'+3,90,3
159,2020-03-07,home,8,S. Finley,yellow,72',72,NaN
3289,2020-03-07,away,1,S. Davies,yellow,88',88,NaN
3290,2020-03-07,away,3,L. Ridehalgh,yellow,79',79,NaN
3298,2020-03-07,away,11,C. Jennings,yellow,34',34,NaN


In [89]:
starters_df[(starters_df['name.shirt'].notna()) & (starters_df['name.shirt'] != starters_df['name.last'])]

,position,shirtNumber,formationPlace,isCaptain,displayName,name.first,name.last,card.type.value,card.type.accessibleValue,card.time.value,card.time.accessibleValue,substitutedOff.periodId,substitutedOff.timeMin,substitutedOff.timeMinSec,substitutedOff.timestamp,substitutedOff.playerOnName,substitutedOff.reason,game_date,team,name.shirt
2751,Defender,4,4,False,Z. Williams,Michael,Issac Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-26,home,Williams
2864,Defender,34,2,False,J. Emmanuel,Joshua,Oluwadurotimi Emmanuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-21,home,Emmanuel
2908,Midfielder,3,3,False,L. Gordon,Liam Spencer,Gordon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-11,home,Liam Gordon
2912,Midfielder,2,2,False,C. Barrett,Connor Ellis,Barrett,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-11,home,Connor Barrett
2918,Defender,33,6,False,I. Ogundere,Isaac Ifeoluwa,Foloronso Olushore Ogundere,NaN,NaN,NaN,NaN,2.0,83.0,82:17,2025-01-18T16:41:12Z,J. Lewis,Tactical,2025-01-18,home,Ogundere
3032,Midfielder,11,2,False,J. Svanþórsson,Jason Daði,Svanþórsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-04,home,Svanthórsson
3047,Goalkeeper,1,1,False,S. Walker,Samuel Colin,Walker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-15,home,Sam Walker
3053,Midfielder,11,7,False,B. Khela,Brandon,Singh Khela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-15,home,Khela
3056,Attacking Midfielder,7,10,False,J. Walker,Jamie Allan,Walker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-15,home,Jamie Walker
3059,Defender,4,4,True,B. Wiredu,Brendan Nana,Akwasi Sarpong-Wiredu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-22,home,Wiredu


In [5]:
data['homeTeam'].keys()

dict_keys(['id', 'urn', 'name', 'alignment', 'formation', 'pitchLayout', 'manager', 'players'])

In [8]:
data['homeTeam']['name']

{'fullName': 'Barrow', 'shortName': 'Barrow', 'code': 'BRW'}

In [10]:
data['homeTeam']['formation']

{'layout': '3511', 'value': '3 - 5 - 1 - 1', 'accessible': '3 - 5 - 1 - 1'}

In [ ]:
# lineup
data['homeTeam']['pitchLayout']

data['homeTeam']['pitchLayout'][0][0].keys()

dict_keys(['shirtNumber', 'name', 'accessibleText'])

In [17]:
data['homeTeam']['manager']

{'id': 'zpqmgnrgccip2xusj9701jh1', 'name': {'full': 'Andy Whing'}}

In [2]:
data['homeTeam']['players'].keys()

NameError: name 'data' is not defined